<a href="https://colab.research.google.com/github/ppraveenhalingali/OASIS-INFOBYTE/blob/main/Car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('car data.csv')

# Display the first 5 rows and column information

print(df.head())
df.tail()



  Car_Name  Year  Selling_Price  Present_Price  Driven_kms Fuel_Type  \
0     ritz  2014           3.35           5.59       27000    Petrol   
1      sx4  2013           4.75           9.54       43000    Diesel   
2     ciaz  2017           7.25           9.85        6900    Petrol   
3  wagon r  2011           2.85           4.15        5200    Petrol   
4    swift  2014           4.60           6.87       42450    Diesel   

  Selling_type Transmission  Owner  
0       Dealer       Manual      0  
1       Dealer       Manual      0  
2       Dealer       Manual      0  
3       Dealer       Manual      0  
4       Dealer       Manual      0  


,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
296,city,2016,9.50,11.6,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.9,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.0,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.5,9000,Diesel,Dealer,Manual,0
300,brio,2016,5.30,5.9,5464,Petrol,Dealer,Manual,0


In [7]:
print("\nDataset Info:")
df.info()

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Driven_kms     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Selling_type   301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB

Missing Values:
Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Driven_kms       0
Fuel_Type        0
Selling_type     0
Transmission     0
Owner            0
dtype: int64


In [8]:
# Assuming the data collection was done around the max year in the dataset (e.g., 2018),
# let's calculate the car's age based on that.
# A simpler and more common approach is to just use a fixed "current" year, e.g., 2020.
current_year = 2020

# Calculate 'Car_Age' and drop the original 'Year' column
df['Car_Age'] = current_year - df['Year']
df.drop('Year', axis=1, inplace=True)

print("Data with new 'Car_Age' feature:")
print(df.head())

Data with new 'Car_Age' feature:
  Car_Name  Selling_Price  Present_Price  Driven_kms Fuel_Type Selling_type  \
0     ritz           3.35           5.59       27000    Petrol       Dealer   
1      sx4           4.75           9.54       43000    Diesel       Dealer   
2     ciaz           7.25           9.85        6900    Petrol       Dealer   
3  wagon r           2.85           4.15        5200    Petrol       Dealer   
4    swift           4.60           6.87       42450    Diesel       Dealer   

  Transmission  Owner  Car_Age  
0       Manual      0        6  
1       Manual      0        7  
2       Manual      0        3  
3       Manual      0        9  
4       Manual      0        6  


In [9]:
# Drop 'Car_Name' as it has too many unique values and is not typically used for simple models.
df.drop('Car_Name', axis=1, inplace=True)

# Convert 'Owner' to string/object type so it is one-hot encoded properly.
# The 'Owner' column is an ordinal feature (0, 1, 3 owners) but we'll treat it as nominal for simplicity
df['Owner'] = df['Owner'].astype('object')

# One-Hot Encoding for categorical variables
categorical_cols = ['Fuel_Type', 'Selling_type', 'Transmission', 'Owner']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print("\nData after One-Hot Encoding:")
print(df.head())


Data after One-Hot Encoding:
   Selling_Price  Present_Price  Driven_kms  Car_Age  Fuel_Type_Diesel  \
0           3.35           5.59       27000        6             False   
1           4.75           9.54       43000        7              True   
2           7.25           9.85        6900        3             False   
3           2.85           4.15        5200        9             False   
4           4.60           6.87       42450        6              True   

   Fuel_Type_Petrol  Selling_type_Individual  Transmission_Manual  Owner_1  \
0              True                    False                 True    False   
1             False                    False                 True    False   
2              True                    False                 True    False   
3              True                    False                 True    False   
4             False                    False                 True    False   

   Owner_3  
0    False  
1    False  
2    False  
3   

In [10]:
from sklearn.model_selection import train_test_split

# Separate features (X) and target (y)
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")


Training set size: 240 samples
Testing set size: 61 samples


In [11]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = rf_model.predict(X_test)

In [12]:
from sklearn.metrics import r2_score, mean_absolute_error

# Calculate R-squared score
r2 = r2_score(y_test, predictions)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)

print("\nModel Evaluation:")
print(f"R-squared Score (R^2): {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f} Lakhs")

# Display the feature importance (which factors matter most)
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
print("\nTop 5 Feature Importances:")
print(feature_importances.nlargest(5))


Model Evaluation:
R-squared Score (R^2): 0.9605
Mean Absolute Error (MAE): 0.6381 Lakhs

Top 5 Feature Importances:
Present_Price          0.883321
Car_Age                0.059274
Driven_kms             0.034717
Transmission_Manual    0.011171
Fuel_Type_Diesel       0.004486
dtype: float64
